<a href="https://colab.research.google.com/github/Blutronics/OffensiveTwitterDetection/blob/Toan/OffensiveDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1236]:
import numpy as np
import pandas as pd
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,r2_score
import xgboost as xgb
import lightgbm as gbm

In [1237]:
data=pd.read_csv("twitter.csv")
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [1238]:
classdf=data[['class','tweet']]
classdf.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [1239]:
tweets=classdf.tweet.to_string()

In [1240]:
tweets = re.sub("[^A-Za-z" "]+"," ", tweets).lower()
tweets = re.sub("[0-9" "]+"," ", tweets)

In [1241]:
class2df=classdf.loc[classdf['class']==2]
class2df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
40,2,""" momma said no pussy cats inside my doghouse """
63,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://..."
66,2,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo..."
67,2,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these..."


In [1242]:
offensivedf=classdf.loc[classdf['class']==1]
offensivedf.head()

,class,tweet
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."


In [1243]:
hatedf=classdf.loc[classdf['class']==0]
hatedf.head()

,class,tweet
85,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1..."
89,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ..."
110,0,"""@DevilGrimz: @VigxRArts you're fucking gay, b..."
184,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL..."
202,0,"""@NoChillPaz: ""At least I'm not a nigger"" http..."


In [1244]:
offensivetweets=offensivedf.tweet.to_string()


In [1245]:
offensivetweets = re.sub("[^A-Za-z" "]+"," ", offensivetweets).lower()
offensivetweets = re.sub("[0-9" "]+"," ", offensivetweets)

In [1246]:
offensivetweets = offensivetweets.split(" ")

In [1247]:
hatespeachtweets=hatedf.tweet.to_string()


In [1248]:
hatespeachtweets = re.sub("[^A-Za-z" "]+"," ", hatespeachtweets).lower()
hatespeachtweets = re.sub("[0-9" "]+"," ", hatespeachtweets)

In [1249]:
hatespeachtweets = hatespeachtweets.split(" ")

In [1250]:
#hatespeachtweets

In [1251]:
vectorizer = TfidfVectorizer()
offensive = vectorizer.fit_transform(offensivetweets)

In [1252]:
with open("stop.txt","r") as sw:
    stop_words = sw.read()

In [1253]:
stop_words = stop_words.split("\n")

In [1254]:
stop_words.extend(["rt","a"])

In [1255]:
offensivetweets = [w for w in offensivetweets if not w in stop_words]


In [1256]:
offensivetweets = " ".join(offensivetweets)

In [1257]:
'''
wordcloud_offensive = WordCloud(
                      background_color='White',
                      width=1800,
                      height=1400
                     ).generate(offensivetweets)'''

"\nwordcloud_offensive = WordCloud(\n                      background_color='White',\n                      width=1800,\n                      height=1400\n                     ).generate(offensivetweets)"

In [1258]:
#plt.imshow(wordcloud_offensive)

In [1259]:
offensivetweets = offensivetweets.split(" ")

In [1260]:
letter_counts = Counter(offensivetweets)
letter_counts.pop('')


1

In [1261]:
most_offensive = letter_counts.most_common(30)
most_offensive

[('bitch', 4601),
 ('bitches', 1784),
 ('hoes', 1377),
 ('pussy', 1059),
 ('hoe', 1027),
 ('don', 714),
 ('fuck', 696),
 ('ass', 619),
 ('nigga', 524),
 ('ain', 505),
 ('shit', 440),
 ('niggas', 418),
 ('lol', 407),
 ('yo', 406),
 ('love', 364),
 ('bad', 347),
 ('http', 298),
 ('im', 282),
 ('hate', 282),
 ('ya', 240),
 ('fucking', 236),
 ('amp', 231),
 ('man', 222),
 ('good', 206),
 ('real', 204),
 ('make', 197),
 ('girl', 197),
 ('niggah', 178),
 ('wit', 176),
 ('wanna', 173)]

In [1262]:
hatespeachtweets = [w for w in hatespeachtweets if not w in stop_words]

In [1263]:
letter_counts2 = Counter(hatespeachtweets)
letter_counts2.pop('')

2

In [1264]:
most_hate = letter_counts2.most_common(30)
most_hate

[('faggot', 102),
 ('bitch', 98),
 ('white', 75),
 ('fuck', 73),
 ('nigga', 60),
 ('ass', 54),
 ('trash', 54),
 ('fag', 51),
 ('niggas', 50),
 ('nigger', 48),
 ('fucking', 46),
 ('hate', 43),
 ('don', 39),
 ('people', 34),
 ('faggots', 31),
 ('bitches', 30),
 ('hoes', 29),
 ('http', 28),
 ('hoe', 25),
 ('shit', 23),
 ('fags', 22),
 ('black', 21),
 ('niggers', 20),
 ('stupid', 20),
 ('pussy', 19),
 ('man', 18),
 ('ugly', 18),
 ('yo', 18),
 ('gay', 17),
 ('kill', 17)]

In [1265]:
def countOccurrences(str, word, listword):
     
    # split the string by spaces in a
    a = str.split(" ")
    # search for pattern in a
    count = 0
    # for i in range(1000):
    for i in range(0, len(a)):
        
        # if match found increase count
        # print(a[i])
      if (word == a[i]):
          count = count + 1
    if count > 0:
      listword.append(1)
    else:
      listword.append(0)
            
    return count, listword

In [1266]:
bitch = []
# for i in range(12):
for i in range(24783):
  count, bitch = countOccurrences(classdf.tweet[i],"bitch", bitch)
  

In [1267]:
bitches = []
# for i in range(12):
for i in range(24783):
  count, bitches = countOccurrences(classdf.tweet[i],"bitches", bitches)
  

In [1268]:
hoes = []
# for i in range(12):
for i in range(24783):
  count, hoes = countOccurrences(classdf.tweet[i],"hoes", hoes)
  

In [1269]:
pussy = []
# for i in range(12):
for i in range(24783):
  count, pussy = countOccurrences(classdf.tweet[i],"pussy", pussy)
  

In [1270]:
hoe = []
# for i in range(12):
for i in range(24783):
  count, hoe = countOccurrences(classdf.tweet[i],"hoe", hoe)
  

In [1271]:
fuck = []
# for i in range(12):
for i in range(24783):
  count, fuck = countOccurrences(classdf.tweet[i],"fuck", fuck)
  

In [1272]:
ass = []
# for i in range(12):
for i in range(24783):
  count, ass = countOccurrences(classdf.tweet[i],"ass", ass)

In [1273]:
niggah = []
# for i in range(12):
for i in range(24783):
  count, niggah = countOccurrences(classdf.tweet[i],"niggah", niggah)

In [1274]:
nigga = []
# for i in range(12):
for i in range(24783):
  count, nigga = countOccurrences(classdf.tweet[i],"nigga", nigga)

In [1275]:
shit = []
# for i in range(12):
for i in range(24783):
  count, shit = countOccurrences(classdf.tweet[i],"shit", shit)

In [1276]:
niggas = []
# for i in range(12):
for i in range(24783):
  count, niggas = countOccurrences(classdf.tweet[i],"niggas", niggas)

In [1277]:
nigger = []
# for i in range(12):
for i in range(24783):
  count, nigger = countOccurrences(classdf.tweet[i],"nigger", nigger)

In [1278]:
gay = []
# for i in range(12):
for i in range(24783):
  count, gay = countOccurrences(classdf.tweet[i],"gay", gay)

In [1279]:
ugly = []
# for i in range(12):
for i in range(24783):
  count, ugly = countOccurrences(classdf.tweet[i],"ugly", ugly)

In [1280]:
#create a new dataframe counting each frequent word for each tweet(row)
#most frequent word= column

In [1281]:
encodedDf = pd.DataFrame(bitch)

In [1282]:
encodedDf

,0
0,0
1,0
2,1
3,0
4,1
...,...
24778,0
24779,0
24780,0
24781,0


In [1283]:
encodedDf.rename(columns = {0:"Bitch"},inplace=True)

In [1284]:
encodedDf["Bitches"] = bitches

In [1285]:
encodedDf["Hoes"] = hoes

In [1286]:
encodedDf["Pussy"] = pussy

In [1287]:
encodedDf["Hoe"] = hoe

In [1288]:
encodedDf["Fuck"] = fuck

In [1289]:
encodedDf["Ass"] = ass

In [1290]:
encodedDf["Nigga"] = nigga

In [1291]:
encodedDf["Shit"] = shit

In [1292]:
encodedDf["Niggas"] = niggas

In [1293]:
encodedDf["Nigger"] = nigger

In [1294]:
#encodedDf["Niggah"] = niggah

In [1295]:
encodedDf["Gay"] = gay

In [1296]:
encodedDf["Ugly"] = ugly

In [1297]:
encodedDf

,Bitch,Bitches,Hoes,Pussy,Hoe,Fuck,Ass,Nigga,Shit,Niggas,Nigger,Gay,Ugly
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0,0,0,0,0,0,0,0,0,0,0,0,0
24779,0,0,0,0,0,0,0,0,0,0,0,0,0
24780,0,0,0,0,0,0,0,0,0,0,0,0,0
24781,0,1,0,0,0,0,0,0,0,0,0,0,0


In [1298]:
faggot = []
for i in range(24783):
  count, faggot = countOccurrences(classdf.tweet[i],"faggot", faggot)
  

In [1299]:
bitch = []
for i in range(24783):
  count, bitch = countOccurrences(classdf.tweet[i],"bitch", bitch)

In [1300]:
fuck = []
for i in range(24783):
  count, fuck = countOccurrences(classdf.tweet[i],"fuck", fuck)

In [1301]:
nigga = []
for i in range(24783):
  count, nigga = countOccurrences(classdf.tweet[i],"nigga", nigga)

In [1302]:
ass = []
for i in range(24783):
  count, ass = countOccurrences(classdf.tweet[i],"ass", ass)

In [1303]:
fag = []
for i in range(24783):
  count, fag = countOccurrences(classdf.tweet[i],"fag", fag)

In [1304]:
niggas = []
for i in range(24783):
  count, niggas = countOccurrences(classdf.tweet[i],"niggas", niggas)

In [1305]:
fucking = []
for i in range(24783):
  count, fucking = countOccurrences(classdf.tweet[i],"fucking", fucking)

In [1306]:
faggots = []
for i in range(24783):
  count, faggots = countOccurrences(classdf.tweet[i],"faggots", faggots)

In [1307]:
white = []
for i in range(24783):
  count, white = countOccurrences(classdf.tweet[i],"white", white)

In [1308]:
trash = []
for i in range(24783):
  count, trash = countOccurrences(classdf.tweet[i],"trash", trash)

In [1309]:
love = []
for i in range(24783):
  count, love = countOccurrences(classdf.tweet[i],"love", love)

In [1310]:
real = []
for i in range(24783):
  count, real = countOccurrences(classdf.tweet[i],"real", real)

In [1311]:
good = []
for i in range(24783):
  count, good = countOccurrences(classdf.tweet[i],"good", good)

In [1312]:
black = []
for i in range(24783):
  count, black = countOccurrences(classdf.tweet[i],"black", black)

In [1313]:

encodedHateDF = pd.DataFrame(faggot)

In [1314]:
encodedHateDF

,0
0,0
1,0
2,0
3,0
4,0
...,...
24778,0
24779,0
24780,0
24781,0


In [1315]:
encodedHateDF.rename(columns = {0: "Faggot"}, inplace = True)

In [1316]:
#encodedHateDF["Bitch"] = bitch

In [1317]:
#encodedHateDF["Fuck"] = fuck

In [1318]:
#encodedHateDF["Nigga"] = nigga

In [1319]:
#encodedHateDF["Ass"] = ass

In [1320]:
encodedHateDF["Fag"] = fag

In [1321]:
#encodedHateDF["Niggas"] = niggas

In [1322]:
encodedHateDF["Fucking"] = fucking

In [1323]:
encodedHateDF["Faggots"] = faggots

In [1324]:
encodedHateDF["White"] = white

In [1325]:
encodedHateDF["Trash"] = trash

In [1326]:
encodedHateDF["Love"] = love

In [1327]:
encodedHateDF["Real"] = real

In [1328]:
encodedHateDF["Good"] = good

In [1329]:
encodedHateDF["Black"] = black

In [1330]:
encodedHateDF

,Faggot,Fag,Fucking,Faggots,White,Trash,Love,Real,Good,Black
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
24778,0,0,0,0,0,1,0,0,0,0
24779,0,0,0,0,0,0,0,0,0,0
24780,0,0,0,0,0,0,0,0,0,0
24781,0,0,0,0,0,0,0,0,0,0


In [1331]:
wordDF = pd.merge(encodedDf, encodedHateDF, left_index=True, right_index=True)

In [1332]:
wordDF.head()

,Bitch,Bitches,Hoes,Pussy,Hoe,Fuck,Ass,Nigga,Shit,Niggas,...,Faggot,Fag,Fucking,Faggots,White,Trash,Love,Real,Good,Black
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [1333]:
target = classdf["class"]

In [1334]:
sm = SMOTE()
X,Y = sm.fit_resample(wordDF,target)

In [1335]:
x_train,x_test,x_target_train,x_target_test = train_test_split(X,Y,test_size=0.2,shuffle=True,random_state=14)

In [1336]:
#rfc=RandomForestClassifier(n_estimators= 250, max_depth=100)
#dtc=DecisionTreeClassifier()
#lr=LogisticRegression(max_iter=600)
#knnc=KNeighborsClassifier()
#svc=SVC()
#nb=GaussianNB()

In [1337]:
#models = [("rfc",rfc),("dtc",dtc),("lr",lr),("knnc",knnc),("svc",svc),("nb",nb)]

In [1338]:
#voting = VotingClassifier(models)

In [1339]:
#voting.fit(x_train,x_target_train)

In [1340]:
#x_pred = voting.predict(x_test)

In [1341]:
#accuracy_score(x_target_test,x_pred)

In [1342]:
#model = xgb.XGBClassifier()
#model.fit(x_train,x_target_train)

In [1343]:
#x_pred = model.predict(x_test)

In [1344]:
#accuracy_score(x_target_test,x_pred)

In [1345]:
model = gbm.LGBMClassifier()

In [1346]:
model.fit(x_train,x_target_train)

LGBMClassifier()

In [1347]:
x_pred = model.predict(x_test)

In [1348]:
accuracy_score(x_target_test,x_pred)

0.7026228938683342